In [5]:
import os
from pydub import AudioSegment
import json
import tqdm

ROOT = r'D:\Download\steam\steam\steamapps\common\Left 4 Dead 2\left4dead2'
VOICE_ROOT = os.path.join(ROOT, 'sound', 'player', 'survivor', 'voice')
CHARACTERS = ['coach', 'gambler', 'mechanic', 'producer']

def read_json(path: str) -> dict:
    with open(path, 'r', encoding='utf-8') as fp:
        data = json.load(fp)
    return data
def write_json(path: str, obj: dict):
    with open(path, 'w', encoding='utf-8') as fp:
        json.dump(obj, fp, ensure_ascii=False, indent=4)

In [6]:
def check_empty(name: str):
    assert name in CHARACTERS
    data = read_json(f'../transcription/{name}_zh.json')
    empty = {}
    for k in data:
        words = data[k].strip()
        if len(words) == 0:
            empty[k] = ''
    if len(empty) > 0:
        empty_path = f'empty.{name}.json'
        write_json(empty_path, empty)
        print('{} unaccomplished words found in {}, write them to {}'.format(len(empty), name, empty_path))
    else:
        print('no unaccomplished words found in {}'.format(name))

for name in CHARACTERS:
    check_empty(name)

no unaccomplished words found in coach
no unaccomplished words found in gambler
no unaccomplished words found in mechanic
no unaccomplished words found in producer


In [4]:
def merge_json(name):
    # merge path2 to path1
    path1 = f'../transcription/{name}_zh.json'
    path2 = f'./empty.{name}.json'

    data1 = read_json(path1)
    data2 = read_json(path2)

    for k in data2:
        data1[k] = data2[k]
        
    # sort
    data1 = {k : data1[k] for k in sorted(data1.keys())}
    write_json(path1, data1)
    os.remove(path2)

for name in CHARACTERS:
    merge_json(name)